In [2]:
import copy

import numpy as np

from cv2 import cv2
from config.Config import Config
from mapping.mapper import Mapper
from agent.agent_handler import AgentHandler
from region_assignment.k_mean_clustring import KMeanClustring
from region_assignment.hungarian_region_assignment import HungarianRegionAssignment
from occupancy_grid_generator.occupancy_grid_generator import OccupancyGridGenerator

# OCCUPANCY GRID GENERATOR

In [3]:
occupancy_grid = OccupancyGridGenerator()

occupancy_grid.generate_occupancy_grid()

# occupancy_grid.show_occupancy_grid_without_obs()
# occupancy_grid.show_occupancy_grid_with_obs()

temp_occupancy_grid_without_obs = occupancy_grid.get_occupancy_grid_without_obs()
temp_occupancy_grid_with_obs = occupancy_grid.get_occupancy_grid_with_obs()

# K-MEAN-CLUSTRING

In [4]:
regions = KMeanClustring(temp_occupancy_grid_without_obs)

regions.find_regions()

temp_regions_xy_points = regions.get_regions_xy_points()
temp_region_centroids = regions.get_centroids()
temp_color_map = regions.get_color_map()
temp_grid_with_regions = copy.copy(regions.get_grid_with_regions())

In [5]:
# regions.show_regions_with_centroids()
# regions.show_regions()

# ind = 0
# for el in temp_regions_xy_points:
#     print("Region", ind, "indices:", el)
#     ind += 1
print("Region centroids:\n", temp_region_centroids)
print("Region color map:\n", temp_color_map)

Region centroids:
 [[786 306]
 [238 306]
 [513 306]]
Region color map:
 [[147, 142, 238], [10, 173, 13], [177, 128, 179]]


# AGENT HANDLER

In [28]:
agenthandler = AgentHandler()

# REGION ASSIGNMENT

In [29]:
cost_matrix = np.zeros(shape=[Config.NO_OF_AGENTS, Config.NO_OF_AGENTS])

print("Agent Positions:")

for i in range(Config.NO_OF_AGENTS):
    temp_agent_pos = agenthandler.get_pos_of_agent(i)
    a = np.array([temp_agent_pos['x'], temp_agent_pos['y']])
    print(a)
    for j in range(len(temp_region_centroids)):
        b = np.array(temp_region_centroids[j])
        cost_matrix[i,j] = np.linalg.norm(a-b)

Agent Positions:
[694  58]
[467 556]
[330  48]


In [30]:
cost_matrix

array([[264.51464988, 519.07610232, 307.02605753],
       [405.2912533 , 339.02949724, 254.19677417],
       [523.9274759 , 273.91239475, 316.31155527]])

In [31]:
hungaian_region_assignment = HungarianRegionAssignment(cost_matrix, temp_grid_with_regions)

hungaian_region_assignment.find_regions()
temp_regions_rows, temp_regions_cols = hungaian_region_assignment.get_regions()
print(temp_regions_rows, temp_regions_cols)
print(hungaian_region_assignment.get_total_cost())

[0 1 2] [0 2 1]
792.6238188051626


In [32]:
hungaian_region_assignment.show_assigned_regions(agenthandler, temp_region_centroids)

In [33]:
print("Region centroids:\n")
ind = 1
for el in temp_region_centroids:
    print("\tRegion:", ind, ", x:", el[0], ", y:", el[1])
    ind += 1
    
print("\nAgent Positions:\n")

for i in range(Config.NO_OF_AGENTS):
    temp_agent_pos = agenthandler.get_pos_of_agent(i)
    print("\tAgent:", i+1, ", x:", temp_agent_pos['x'], ", y:", temp_agent_pos['y'])
    
print("\nCost Matrix:\n\n", cost_matrix, "\n")

ind = 1
for el in temp_regions_cols:
    print("Region Assigned to Agent: {} is Region {}".format(ind, el))
    ind += 1
    
print("\nTotal cost:", hungaian_region_assignment.get_total_cost())

Region centroids:

	Region: 1 , x: 786 , y: 306
	Region: 2 , x: 238 , y: 306
	Region: 3 , x: 513 , y: 306

Agent Positions:

	Agent: 1 , x: 694 , y: 58
	Agent: 2 , x: 467 , y: 556
	Agent: 3 , x: 330 , y: 48

Cost Matrix:

 [[264.51464988 519.07610232 307.02605753]
 [405.2912533  339.02949724 254.19677417]
 [523.9274759  273.91239475 316.31155527]] 

Region Assigned to Agent: 1 is Region 0
Region Assigned to Agent: 2 is Region 2
Region Assigned to Agent: 3 is Region 1

Total cost: 792.6238188051626


# GRID MAPPER

In [27]:
grid_mapper = Mapper(global_grid=temp_occupancy_grid_with_obs)

display = copy.copy(temp_occupancy_grid_with_obs)
font = cv2.FONT_HERSHEY_SIMPLEX

while(1):

    display = copy.copy(temp_occupancy_grid_with_obs)

    for i in range(Config.NO_OF_AGENTS):

        temp_pos = agenthandler.get_pos_of_agent(i)

        temp_pos_x = temp_pos['x']
        temp_pos_y = temp_pos['y']

        if i == 0:
            color_b,color_g,color_r = 255,0,0
        elif i == 1:
            color_b,color_g,color_r = 0,255,0
        else:
            color_b,color_g,color_r = 0,0,255

        cv2.circle(temp_occupancy_grid_with_obs, (temp_pos_x, temp_pos_y), 1, (color_b,color_g,color_r), -1)

        img = cv2.ellipse(display,(temp_pos_x,temp_pos_y),(10,10),0,15,345,(color_b,color_g,color_r),-1)

        grid_mapper.map_grid(agent_no=i, agent_pos=temp_pos)
    
    temp_img = cv2.resize(img, (512, 512))

    mapped_grid = grid_mapper.get_mapped_grid()
    
    temp_mapped_img = cv2.resize(mapped_grid, (512, 512))

    numpy_horizontal = np.hstack((temp_img, temp_mapped_img))

    cv2.imshow('image', numpy_horizontal)
    k = cv2.waitKey(1) & 0xFF

    if k == ord('q'):
        break
    if k == ord('1'):
        agent_number = 0
    if k == ord('2'):
        agent_number = 1
    if k == ord('3'):
        agent_number = 2
    if k == ord('w'):
        agenthandler.move_agent(agent_number, 0, -5)
    if k == ord('s'):
        agenthandler.move_agent(agent_number, 0, 5)
    if k == ord('a'):
        agenthandler.move_agent(agent_number, -5, 0)
    if k == ord('d'):
        agenthandler.move_agent(agent_number, 5, 0)


cv2.destroyAllWindows()
# grid_mapper.show_mapped_grid()

# cv2.imshow('Occupancy_grid',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [25]:
# black_img = np.zeros(shape=[512,512,3], dtype=np.uint8)
# font = cv2.FONT_HERSHEY_SIMPLEX
# img = cv2.putText(black_img,'OpenCV',(10,500), font, 4,(255,255,255),2,cv2.LINE_AA)

# cv2.imshow('Ellipse', img)

# cv2.waitKey(0)
# cv2.destroyAllWindows()